In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_1h.parquet")
label_side = np.load("data/label_side_1h.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(44867, 11841)
(44867, 7)


4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_ac_23                        0
1h_ac_22                        0
1h_mod_stochastic               0
1h_trendflex                    0
1h_ac_31                        0
Length: 11841, dtype: int64

In [3]:
df_features = df_features[364:]
label_side = label_side[364:]

print(df_features.shape)
print(label_side.shape)

label_side

(44503, 11841)
(44503, 7)


array([[ 1.58666418e+12,  6.81173000e+03,  6.91519000e+03, ...,
         6.77566000e+03,  7.38005150e+04,  1.00000000e+00],
       [ 1.58670006e+12,  6.91519000e+03,  7.08500000e+03, ...,
         6.90500000e+03,  7.22904840e+04,  1.00000000e+00],
       [ 1.58670972e+12,  7.08500000e+03,  7.11225000e+03, ...,
         7.04632000e+03,  7.13427380e+04,  1.00000000e+00],
       ...,
       [ 1.73826948e+12,  1.05165000e+05,  1.04909800e+05, ...,
         1.04768000e+05,  4.98372300e+03, -1.00000000e+00],
       [ 1.73827092e+12,  1.04909800e+05,  1.05189300e+05, ...,
         1.04737000e+05,  4.83362100e+03, -1.00000000e+00],
       [ 1.73827926e+12,  1.05189200e+05,  1.04416800e+05, ...,
         1.04356100e+05,  4.86977900e+03, -1.00000000e+00]])

In [4]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([20772, 23731]))
(array([0, 1]), array([3872, 5029]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-16 11:14:49,612] A new study created in memory with name: no-name-3da64ca0-4a49-44cc-9b7f-a8649f702ae6
[I 2025-03-16 11:15:08,285] Trial 0 finished with value: 0.8178912000479861 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 37, 'max_depth': 541, 'min_gain_to_split': 0.2075842094800597, 'min_data_in_leaf': 31, 'lambda_l1': 46.43581053326298, 'lambda_l2': 67.2746074393392, 'num_boost_round': 1350}. Best is trial 0 with value: 0.8178912000479861.


precision: 0.7925812115891132, recall: 0.7180353947106781, f1: 0.7534689619196662


[I 2025-03-16 11:15:16,875] Trial 1 finished with value: 0.824629904816527 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 142, 'max_depth': 729, 'min_gain_to_split': 0.7509966141756917, 'min_data_in_leaf': 161, 'lambda_l1': 81.21786573900489, 'lambda_l2': 39.8011240307943, 'num_boost_round': 538}. Best is trial 1 with value: 0.824629904816527.


precision: 0.7939160239931449, recall: 0.7369258301849274, f1: 0.764360111374652


[I 2025-03-16 11:16:02,254] Trial 2 finished with value: 0.8234512554456878 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 87, 'max_depth': 68, 'min_gain_to_split': 0.7946634354370475, 'min_data_in_leaf': 102, 'lambda_l1': 64.139533546089, 'lambda_l2': 14.833031886003083, 'num_boost_round': 1078}. Best is trial 1 with value: 0.824629904816527.


...

precision: 0.7971728421503534, recall: 0.7401073772121695, f1: 0.7675809445246442


[I 2025-03-16 11:38:14,269] Trial 98 finished with value: 0.8318371729095215 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 39, 'max_depth': 754, 'min_gain_to_split': 0.25728191055561483, 'min_data_in_leaf': 76, 'lambda_l1': 68.65172509870163, 'lambda_l2': 62.43730247190459, 'num_boost_round': 181}. Best is trial 71 with value: 0.835597799293026.


precision: 0.791412334245422, recall: 0.7476635514018691, f1: 0.7689161554192229


[I 2025-03-16 11:38:29,109] Trial 99 finished with value: 0.8271833797856728 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 163, 'max_depth': 915, 'min_gain_to_split': 0.4142655528481294, 'min_data_in_leaf': 152, 'lambda_l1': 17.92261143957211, 'lambda_l2': 90.01970946656233, 'num_boost_round': 204}. Best is trial 71 with value: 0.835597799293026.


precision: 0.7896799477465709, recall: 0.7212169417379201, f1: 0.7538973186447724


In [6]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 173,
 'max_depth': 819,
 'min_gain_to_split': 0.3351703543922414,
 'min_data_in_leaf': 440,
 'lambda_l1': 34.80933606498479,
 'lambda_l2': 85.0208819534468,
 'num_boost_round': 218}

In [7]:
params = {
    # "is_unbalance": True,
    **study.best_params,
}

dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

In [8]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
import plotly.express as px

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8336102567916005


In [10]:
model.save_model("custom_indicators/models/model_side.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_1h.parquet")
meta_label = np.load("data/label_meta_1h.npy")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features.isna().sum(axis=0).sort_values(ascending=False)

4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_comb_spectrum_pwr_14         0
1h_comb_spectrum_pwr_13         0
1h_comb_spectrum_pwr_12         0
1h_comb_spectrum_pwr_11         0
side_model_res                  0
Length: 11842, dtype: int64

In [2]:
df_features = df_features[364:]
meta_label = meta_label[364:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

(44503, 11842)
(44503, 7)


In [3]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

np.unique(meta_label_test, return_counts=True)

(44503, 2368)


(array([0, 1]), array([1890, 7011]))

In [10]:
import lightgbm as lgb
import optuna
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    fbeta_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 500),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.2, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba)
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-03-11 23:25:16,558] A new study created in memory with name: no-name-9e520850-07ab-499c-9300-d1c8d07d6213
[I 2025-03-11 23:26:05,251] Trial 0 finished with value: 0.7540651097349406 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 369, 'max_depth': 476, 'min_gain_to_split': 0.212351622639555, 'min_data_in_leaf': 121, 'lambda_l1': 2.984838941621447, 'lambda_l2': 49.515945603639494, 'num_boost_round': 999}. Best is trial 0 with value: 0.7540651097349406.


precision: 0.8265873015873015, recall: 0.8913136499786051, f1: 0.8577311097385217, auc: 0.7747298085623575, fbeta: 0.7540651097349406


[I 2025-03-11 23:27:29,389] Trial 1 finished with value: 0.7401162936679861 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 284, 'max_depth': 451, 'min_gain_to_split': 0.21121406718502314, 'min_data_in_leaf': 276, 'lambda_l1': 56.22829363692332, 'lambda_l2': 29.3047030792834, 'num_boost_round': 1355}. Best is trial 0 with value: 0.7540651097349406.


precision: 0.8791390728476821, recall: 0.7573812580231065, f1: 0.8137307486016397, auc: 0.7747672402928429, fbeta: 0.7401162936679861


[I 2025-03-11 23:28:05,924] Trial 2 finished with value: 0.7209619355868551 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 399, 'max_depth': 271, 'min_gain_to_split': 0.06630234901642303, 'min_data_in_leaf': 72, 'lambda_l1': 13.627870920995734, 'lambda_l2': 89.59158455811848, 'num_boost_round': 893}. Best is trial 0 with value: 0.7540651097349406.


...

precision: 0.8473918666475069, recall: 0.8411068321209528, f1: 0.8442376521116679, auc: 0.7759218129636043, fbeta: 0.7565158441390865


[I 2025-03-12 00:39:29,716] Trial 98 finished with value: 0.7569549574422334 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 326, 'max_depth': 317, 'min_gain_to_split': 0.06780209931703648, 'min_data_in_leaf': 91, 'lambda_l1': 13.471623764267047, 'lambda_l2': 55.144296113156045, 'num_boost_round': 1507}. Best is trial 23 with value: 0.7592910162422741.


precision: 0.8378004973749654, recall: 0.8649265440022821, f1: 0.8511474489437856, auc: 0.7759285295442763, fbeta: 0.7569549574422334


[I 2025-03-12 00:42:04,166] Trial 99 finished with value: 0.7476349004515974 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 263, 'max_depth': 357, 'min_gain_to_split': 0.13100565829623037, 'min_data_in_leaf': 90, 'lambda_l1': 47.18609881415287, 'lambda_l2': 43.45722316581008, 'num_boost_round': 1714}. Best is trial 23 with value: 0.7592910162422741.


precision: 0.866614420062696, recall: 0.7886178861788617, f1: 0.8257785079531028, auc: 0.7757477856037263, fbeta: 0.7476349004515974


In [11]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 324,
 'max_depth': 367,
 'min_gain_to_split': 0.2515399909199958,
 'min_data_in_leaf': 156,
 'lambda_l1': 11.69499289726129,
 'lambda_l2': 42.941306776998665,
 'num_boost_round': 476}

In [12]:
params = {
    "is_unbalance": True,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

In [13]:
import plotly.express as px

res = model.predict(meta_features_test)

fig = px.histogram(res, nbins=100)
fig.show()

In [14]:
model.save_model("custom_indicators/models/model_meta.txt")